In [3]:
from eth_abi.abi import encode
import json

# Input the label in OLI format:
#   chain_id should follow the CAIP-2 standard
#   contract_address should be the address of the contract you want to label
#   tags_json should be a JSON string
chain_id = 'eip:8453'
contract_address = '0x498581ff718922c3f8e6a244956af099b2652b2b'
tags_json = json.dumps({
                'is_eoa': False, 
                'is_proxy': False,
                'contract_name': 'Pool Manager v4',
                'deployment_tx': '0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349',
                'deployer_address': '0x2179a60856E37dfeAacA0ab043B931fE224b27B6',
                'owner_project': 'uniswap',
                'deployment_date': '2025-01-21 20:28:43',
                'version': 4,
                'source_code_verified': 'https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/'
            }, separators=(',', ':'))

# ABI encode
encoded_data = encode(['string', 'string'], [chain_id, tags_json])
data = f"0x{encoded_data.hex()}"

print(data)

0x0000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000086569703a3834353300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001a27b2269735f656f61223a66616c73652c2269735f70726f7879223a66616c73652c22636f6e74726163745f6e616d65223a22506f6f6c204d616e61676572207634222c226465706c6f796d656e745f7478223a22307832356634383266626439346364656131316230313837333265343535623865396139343062393333636162646533633063356464363365613635653835333439222c226465706c6f7965725f61646472657373223a22307832313739613630383536453337646665416163413061623034334239333166453232346232374236222c226f776e65725f70726f6a656374223a22756e6973776170222c226465706c6f796d656e745f64617465223a22323032352d30312d32312032303a32383a3433222c2276657273696f6e223a342c22736f757263655f636f64655f7665726966696564223a2268747470733a2f2f7265706f2e736f757263696679

In [ ]:
from web3 import Web3
from eth_abi.abi import encode
import json
import secrets
import time
import hashlib
from eas_contracts import eas_address, eas_abi

# Connect to an Ethereum node
rpc = 'https://sepolia.base.org'
web3 = Web3(Web3.HTTPProvider(rpc))

# Account to sign the attestation
address = ''
private_key = ''

# Check if connected to the node
if not web3.is_connected():
    raise Exception("Failed to connect to the Ethereum node")

# Label Pool Schema v1.0.0
schema = '0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68'

# Create a random salt (32 bytes)
salt = f"0x{secrets.token_hex(32)}"

# Current time in seconds
current_time = int(time.time())

# Create a simplified EIP-712 structure
typed_data = {
    "types": {
        "EIP712Domain": [
            {"name": "name", "type": "string"},
            {"name": "version", "type": "string"},
            {"name": "chainId", "type": "string"},
            {"name": "verifyingContract", "type": "address"}
        ],
        "Attest": [
            {"name": "version", "type": "uint16"},
            {"name": "schema", "type": "bytes32"},
            {"name": "recipient", "type": "address"},
            {"name": "time", "type": "uint64"},
            {"name": "expirationTime", "type": "uint64"},
            {"name": "revocable", "type": "bool"},
            {"name": "refUID", "type": "bytes32"},
            {"name": "data", "type": "bytes"},
            {"name": "salt", "type": "bytes32"}
        ]
    },
    "primaryType": "Attest",
    "domain": {
        "name": "EAS Attestation",
        "version": "1.2.0",
        "chainId": "84532",  # Base Sepolia chainId in string format
        "verifyingContract": eas_address
    },
    "message": {
        "version": 2,
        "schema": schema,
        "recipient": contract_address,
        "time": str(current_time),
        "expirationTime": "0",
        "revocable": True,
        "refUID": "0x0000000000000000000000000000000000000000000000000000000000000000",
        "data": data,
        "salt": salt
    }
}

###### Generate a hash of the typed data according to EIP-712 (????)


from eth_account import Account
from eth_account.messages import encode_defunct

'''
# Convert the hash to the format expected by eth_account
message_to_sign = encode_defunct(primitive=message_hash)

# Sign the message with a local private key
signed_message = Account.sign_message(message_to_sign, private_key=PRIVATE_KEY)

# Extract r, s, v
r = signed_message.r.to_bytes(32, 'big')
s = signed_message.s.to_bytes(32, 'big')
v = signed_message.v

'''

# Calculate UID ????? wrong????
def get_offchain_uid(schema, recipient, time_value, expiration_time, revocable, ref_uid, data_hex, salt_hex):
    # Convert parameters to appropriate types
    encoded = encode(
        ['uint16', 'bytes32', 'address', 'uint64', 'uint64', 'bool', 'bytes32', 'bytes', 'bytes32'],
        [
            2,  # version
            bytes.fromhex(schema.replace('0x', '')),
            Web3.to_checksum_address(recipient),
            int(time_value),
            int(expiration_time),
            revocable,
            bytes.fromhex(ref_uid.replace('0x', '')),
            bytes.fromhex(data_hex.replace('0x', '')),
            bytes.fromhex(salt_hex.replace('0x', ''))
        ]
    )
    return "0x" + Web3.keccak(encoded).hex()

uid = get_offchain_uid(
    schema,
    contract_address,
    typed_data["message"]["time"],
    typed_data["message"]["expirationTime"],
    typed_data["message"]["revocable"],
    typed_data["message"]["refUID"],
    typed_data["message"]["data"],
    typed_data["message"]["salt"]
)


Signing error: {'code': -32000, 'message': 'unknown account'}
Note: web3.eth.sign requires an unlocked account in your Ethereum node.
If using a remote node or cannot unlock your account, you need to implement local signing.
Implementing fallback signing...
{
  "sig": {
    "domain": {
      "name": "EAS Attestation",
      "version": "1.2.0",
      "chainId": "84532",
      "verifyingContract": "0x4200000000000000000000000000000000000021"
    },
    "primaryType": "Attest",
    "types": {
      "EIP712Domain": [
        {
          "name": "name",
          "type": "string"
        },
        {
          "name": "version",
          "type": "string"
        },
        {
          "name": "chainId",
          "type": "string"
        },
        {
          "name": "verifyingContract",
          "type": "address"
        }
      ],
      "Attest": [
        {
          "name": "version",
          "type": "uint16"
        },
        {
          "name": "schema",
          "type": "bytes

In [ ]:
import requests

# Format the attestation & send to API post endpoint
signed_attestation = {
    "sig": {
        "domain": typed_data["domain"],
        "primaryType": typed_data["primaryType"],
        "types": typed_data["types"],
        "message": typed_data["message"],
        "uid": uid,
        "version": 2,
        "signature": {
            "r": "0x" + r.hex(),
            "s": "0x" + s.hex(),
            "v": v
        }
    },
    "signer": address
}

# Convert to JSON
payload_json = json.dumps(signed_attestation, separators=(',', ':'))
print('the payload_json is:', payload_json)

# Prepare data for the API endpoint
api_payload = {
    "filename": "eas.txt",
    "textJson": payload_json
}

# Send the data to the EAS API endpoint
api_url = "https://base-sepolia.easscan.org/offchain/store"
headers = {
    "Content-Type": "application/json"
}

try:
    response = requests.post(api_url, json=api_payload, headers=headers)
    
    # Print the response
    print("Status Code:", response.status_code)
    print("Response:")
    print(response.text)
    
    # If the response is JSON, pretty print it
    try:
        print("\nJSON Response:")
        print(json.dumps(response.json(), indent=2))
    except:
        pass
    
except Exception as e:
    print(f"Error sending data to API: {e}")

# Below here more testing

In [ ]:
import json

data_dict = {
    "sig": {
        "version": 2,
        "uid": "0x38ee1f78da420a4cdd54f3b151505c99343f85cdd10df08a4faf5eaf8e62adf0",
        "domain": {
            "name": "EAS Attestation",
            "version": "1.2.0",
            "chainId": "84532",
            "verifyingContract": "0x4200000000000000000000000000000000000021"
        },
        "primaryType": "Attest",
        "message": {
            "version": 2,
            "recipient": "0x498581ff718922c3f8e6a244956af099b2652b2b",
            "expirationTime": "0",
            "time": "1740486638",
            "revocable": True,
            "schema": "0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68",
            "refUID": "0x0000000000000000000000000000000000000000000000000000000000000000",
            "data": "0x0000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000086569703a3834353300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001a27b2269735f656f61223a66616c73652c2269735f70726f7879223a66616c73652c22636f6e74726163745f6e616d65223a22506f6f6c204d616e61676572207634222c226465706c6f796d656e745f7478223a22307832356634383266626439346364656131316230313837333265343535623865396139343062393333636162646533633063356464363365613635653835333439222c226465706c6f7965725f61646472657373223a22307832313739613630383536453337646665416163413061623034334239333166453232346232374236222c226f776e65725f70726f6a656374223a22756e6973776170222c226465706c6f796d656e745f64617465223a22323032352d30312d32312032303a32383a3433222c2276657273696f6e223a342c22736f757263655f636f64655f7665726966696564223a2268747470733a2f2f7265706f2e736f7572636966792e6465762f636f6e7472616374732f7061727469616c5f6d617463682f383435332f3078343938353831664637313839323263336638653641323434393536614630393942323635326232622f227d000000000000000000000000000000000000000000000000000000000000",
            "salt": "0x09afad15359a474404c00f8df52b21e600530ab51f089588348c299b90a56b1e"
        },
        "types": {"Attest": []},  # Replaced [Array] with empty list
        "signature": {
            "v": 27,
            "r": "0x54b5608d513c7ae19a57540f47a0a6e1adfde0a250d6b369f102f68cf4946a62",
            "s": "0x4136861cc7ac9f69bca44412fb76d35f0ad74987d265076818d391efa8a8fe11"
        }
    },
    "signer": "0x732D31D49467c08F41fD0727537995ea45dD4Ba7"
}


In [76]:
typed_data

{'types': {'EIP712Domain': [{'name': 'name', 'type': 'string'},
   {'name': 'version', 'type': 'string'},
   {'name': 'chainId', 'type': 'string'},
   {'name': 'verifyingContract', 'type': 'address'}],
  'Attest': [{'name': 'version', 'type': 'uint16'},
   {'name': 'schema', 'type': 'bytes32'},
   {'name': 'recipient', 'type': 'address'},
   {'name': 'time', 'type': 'uint64'},
   {'name': 'expirationTime', 'type': 'uint64'},
   {'name': 'revocable', 'type': 'bool'},
   {'name': 'refUID', 'type': 'bytes32'},
   {'name': 'data', 'type': 'bytes'},
   {'name': 'salt', 'type': 'bytes32'}]},
 'primaryType': 'Attest',
 'domain': {'name': 'EAS Attestation',
  'version': '1.2.0',
  'chainId': '84532',
  'verifyingContract': '0x4200000000000000000000000000000000000021'},
 'message': {'version': 2,
  'schema': '0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68',
  'recipient': '0x498581ff718922c3f8e6a244956af099b2652b2b',
  'time': '1740486638',
  'expirationTime': '0',
  'revoca

In [ ]:
from eip712.messages import EIP712Message, EIP712Type
from eth_account import Account
import json

# Define the EIP712 types based on your data
class Attest(EIP712Type):
    version: "uint16"
    schema: "bytes32"
    recipient: "address"
    time: "uint64"
    expirationTime: "uint64"
    revocable: "bool"
    refUID: "bytes32"
    data: "bytes"
    salt: "bytes32"

# Define the EIP712 message class with your domain parameters
class EASAttestation(EIP712Message):
    _name_ = "EAS Attestation"
    _version_ = "1.2.0"
    _chainId_ = 84532  # Note: converted from string to int
    _verifyingContract_ = "0x4200000000000000000000000000000000000021"
    
    # Define the message field
    attest: Attest

# Create an instance with your message data
message_instance = EASAttestation(
    attest=Attest(
        version=2,
        schema="0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68",
        recipient="0x498581ff718922c3f8e6a244956af099b2652b2b",
        time=1740486638,  # Note: converted from string to int
        expirationTime=0,  # Note: converted from string to int
        revocable=True,
        refUID="0x0000000000000000000000000000000000000000000000000000000000000000",
        data="0x0000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000086569703a3834353300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001a27b2269735f656f61223a66616c73652c2269735f70726f7879223a66616c73652c22636f6e74726163745f6e616d65223a22506f6f6c204d616e61676572207634222c226465706c6f796d656e745f7478223a22307832356634383266626439346364656131316230313837333265343535623865396139343062393333636162646533633063356464363365613635653835333439222c226465706c6f7965725f61646472657373223a22307832313739613630383536453337646665416163413061623034334239333166453232346232374236222c226f776e65725f70726f6a656374223a22756e6973776170222c226465706c6f796d656e745f64617465223a22323032352d30312d32312032303a32383a3433222c2276657273696f6e223a342c22736f757263655f636f64655f7665726966696564223a2268747470733a2f2f7265706f2e736f7572636966792e6465762f636f6e7472616374732f7061727469616c5f6d617463682f383435332f3078343938353831664637313839323263336638653641323434393536614630393942323635326232622f227d000000000000000000000000000000000000000000000000000000000000",
        salt="0x09afad15359a474404c00f8df52b21e600530ab51f089588348c299b90a56b1e"
    )
)

# Get the signable message
signable_message = message_instance.signable_message

# Sign the message
signed = Account.sign_message(signable_message, private_key=private_key)

# Extract signature components
signature = {
    "r": f"0x{signed.r.to_bytes(32, 'big').hex()}",
    "s": f"0x{signed.s.to_bytes(32, 'big').hex()}",
    "v": signed.v
}

# Full signature as hex
hex_signature = signed.signature.hex()
signature

{'r': '0x707fc7c980a2e53b6fe88f0eb840312e9cb85526d820216d151a32977d12f354',
 's': '0x6915f44fd7f9292195b00ea06f38790576ba96cbac57af63a9b5992fd1774612',
 'v': 27}

In [107]:
data_dict["sig"]["message"]["data"]  == data

True

In [108]:
print(data_dict["sig"]["message"]["data"])
print(data)

0x0000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000086569703a3834353300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001a27b2269735f656f61223a66616c73652c2269735f70726f7879223a66616c73652c22636f6e74726163745f6e616d65223a22506f6f6c204d616e61676572207634222c226465706c6f796d656e745f7478223a22307832356634383266626439346364656131316230313837333265343535623865396139343062393333636162646533633063356464363365613635653835333439222c226465706c6f7965725f61646472657373223a22307832313739613630383536453337646665416163413061623034334239333166453232346232374236222c226f776e65725f70726f6a656374223a22756e6973776170222c226465706c6f796d656e745f64617465223a22323032352d30312d32312032303a32383a3433222c2276657273696f6e223a342c22736f757263655f636f64655f7665726966696564223a2268747470733a2f2f7265706f2e736f757263696679

In [58]:
'{"is_eoa":false,"is_proxy":false,"contract_name":"Pool Manager v4","deployment_tx":"0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349","deployer_address":"0x2179a60856E37dfeAacA0ab043B931fE224b27B6","owner_project":"uniswap","deployment_date":"2025-01-21 20:28:43","version":4,"source_code_verified":"https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/"}' == '{"is_eoa":"False","is_proxy":"False","contract_name":"Pool Manager v4","deployment_tx":"0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349","deployer_address":"0x2179a60856E37dfeAacA0ab043B931fE224b27B6","owner_project":"uniswap","deployment_date":"2025-01-21 20:28:43","version":4,"source_code_verified":"https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/"}'

False

In [ ]:
'{"is_eoa":false,"is_proxy":false,"contract_name":"Pool Manager v4","deployment_tx":"0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349","deployer_address":"0x2179a60856E37dfeAacA0ab043B931fE224b27B6","owner_project":"uniswap","deployment_date":"2025-01-21 20:28:43","version":4,"source_code_verified":"https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/"}'
'{"is_eoa":false,"is_proxy":false,"contract_name":"Pool Manager v4","deployment_tx":"0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349","deployer_address":"0x2179a60856E37dfeAacA0ab043B931fE224b27B6","owner_project":"uniswap","deployment_date":"2025-01-21 20:28:43","version":4,"source_code_verified":"https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/"}'

'{"is_eoa":"False","is_proxy":"False","contract_name":"Pool Manager v4","deployment_tx":"0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349","deployer_address":"0x2179a60856E37dfeAacA0ab043B931fE224b27B6","owner_project":"uniswap","deployment_date":"2025-01-21 20:28:43","version":4,"source_code_verified":"https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/"}'

In [ ]:
from eip712.messages import EIP712Message, EIP712Type
from eth_account import Account
import json

schema = '0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68'
recipient = '0x498581ff718922c3f8e6a244956af099b2652b2b'
data = '0x0000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000086569703a3834353300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001a27b2269735f656f61223a66616c73652c2269735f70726f7879223a66616c73652c22636f6e74726163745f6e616d65223a22506f6f6c204d616e61676572207634222c226465706c6f796d656e745f7478223a22307832356634383266626439346364656131316230313837333265343535623865396139343062393333636162646533633063356464363365613635653835333439222c226465706c6f7965725f61646472657373223a22307832313739613630383536453337646665416163413061623034334239333166453232346232374236222c226f776e65725f70726f6a656374223a22756e6973776170222c226465706c6f796d656e745f64617465223a22323032352d30312d32312032303a32383a3433222c2276657273696f6e223a342c22736f757263655f636f64655f7665726966696564223a2268747470733a2f2f7265706f2e736f7572636966792e6465762f636f6e7472616374732f7061727469616c5f6d617463682f383435332f3078343938353831664637313839323263336638653641323434393536614630393942323635326232622f227d000000000000000000000000000000000000000000000000000000000000'

# Define the EIP712 type to match the contract's expected structure
class AttestRequest(EIP712Type):
    schema: "bytes32"
    recipient: "address"
    expirationTime: "uint256"  # Changed from uint64 to uint256
    revocable: "bool"
    refUID: "bytes32"
    data: "bytes"
    value: "uint256"  # Added value field to match the contract

# Define the EIP712 message class with your domain parameters
class EASAttestation(EIP712Message):
    _name_ = "EAS Attestation"
    _version_ = "1.2.0"
    _chainId_ = 84532
    _verifyingContract_ = "0x4200000000000000000000000000000000000021"
    
    # Define the message field to match contract expectations
    request: AttestRequest

# Create an instance with your message data matching the contract parameters
message_instance = EASAttestation(
    request=AttestRequest(
        schema=schema,
        recipient=recipient,
        expirationTime=0,
        revocable=True,
        refUID="0x0000000000000000000000000000000000000000000000000000000000000000",
        data=data,
        value=0  # Added to match the contract function
    )
)

# Get the signable message
signable_message = message_instance.signable_message

# Sign the message
signed = Account.sign_message(signable_message, private_key=private_key)

# Extract signature components
signature = {
    "r": f"0x{signed.r.to_bytes(32, 'big').hex()}",
    "s": f"0x{signed.s.to_bytes(32, 'big').hex()}",
    "v": signed.v
}

# Create the JSON payload structure matching your original format but with updated fields
payload = {
    "sig": {
        "domain": {
            "name": "EAS Attestation",
            "version": "1.2.0",
            "chainId": "84532",
            "verifyingContract": "0x4200000000000000000000000000000000000021"
        },
        "primaryType": "AttestRequest",  # Updated to match our type
        "message": {
            "schema": schema,
            "recipient": recipient,
            "expirationTime": "0",  # String representation
            "revocable": True,
            "refUID": "0x0000000000000000000000000000000000000000000000000000000000000000",
            "data": data,
            "value": "0"  # String representation
        },
        "types": {
            "AttestRequest": [
                {"name": "schema", "type": "bytes32"},
                {"name": "recipient", "type": "address"},
                {"name": "expirationTime", "type": "uint256"},
                {"name": "revocable", "type": "bool"},
                {"name": "refUID", "type": "bytes32"},
                {"name": "data", "type": "bytes"},
                {"name": "value", "type": "uint256"}
            ]
        },
        "signature": {
            "v": signed.v,
            "r": signature["r"],
            "s": signature["s"]
        }
    },
    "signer": Account.from_key(private_key).address
}

# Convert to JSON string
payload_json = json.dumps(payload)
print(payload_json)